# Simplified Confident Learning
*Author: Curtis G. Northcutt, cgn@mit.edu*

In this tutorial, we show how to implement confident learning without using cleanlab (for the most part). 
This tutorial is to confident learning what this tutorial https://pytorch.org/tutorials/beginner/examples_tensor/two_layer_net_numpy.html
is to deep learning.

The actual implementations in cleanlab are complex because they support parallel processing, numerous type and input checks, lots of hyper-parameter settings, lots of utilities to make things work smoothly for all types of inputs, and ancillary functions.

I ignore all of that here and provide you a bare-bones implementation using mostly for-loops and some numpy.
Here we'll do two simple things:
1. Compute the confident joint which fully characterizes all label noise.
2. Find the indices of all label errors, ordered by likelihood of being an error.

## INPUT (stuff we need beforehand):
1. labels - These are the noisy labels. This is an np.array of noisy labels, shape (n,1)
2. pred_probs - These are the out-of-sample holdout predicted probabilities for every example in your dataset. This is an np.array (2d) of probabilities, shape (n, m)

## OUTPUT (what this returns):
1. confident_joint - an (m, m) np.array matrix characterizing all the label error counts for every pair of labels.
2. label_errors_idx - a numpy array comprised of indices of every label error, ordered by likelihood of being a label error.

In this tutorial we use the handwritten digits dataset as an example.

In [1]:
import cleanlab
import numpy as np
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression

# To silence convergence warnings caused by using a weak
# logistic regression classifier on image data
import warnings

warnings.simplefilter("ignore")
np.random.seed(477)


In [2]:
# STEP 0 - Get some real digits data. Add a bunch of label errors. Get probs.

# Get handwritten digits data
X = load_digits()["data"]
y = load_digits()["target"]
print("Handwritten digits datasets number of classes:", len(np.unique(y)))
print("Handwritten digits datasets number of examples:", len(y))

# Add lots of errors to labels
NUM_ERRORS = 100
labels = np.array(y)
error_indices = np.random.choice(len(labels), NUM_ERRORS, replace=False)
for i in error_indices:
    # Switch to some wrong label thats a different class
    wrong_label = np.random.choice(np.delete(range(10), labels[i]))
    labels[i] = wrong_label

# Confirm that we indeed added NUM_ERRORS label errors
assert len(labels) - sum(labels == y) == NUM_ERRORS
actual_label_errors = np.arange(len(y))[labels != y]
print("\nIndices of actual label errors:\n", actual_label_errors)

# To keep the tutorial short, we use cleanlab to get the
# out-of-sample predicted probabilities using cross-validation
# with a very simple, non-optimized logistic regression classifier
pred_probs = cleanlab.count.estimate_cv_predicted_probabilities(
    X, labels, clf=LogisticRegression(max_iter=1000, multi_class="auto", solver="lbfgs")
)

# Now we have our noisy labels `labels` and predicted probabilities `pred_probs`.
# That's all we need for confident learning.


Handwritten digits datasets number of classes: 10
Handwritten digits datasets number of examples: 1797

Indices of actual label errors:
 [  24   27   42   47   50   54   58   62   85   94  105  107  110  157
  159  182  201  211  220  223  232  300  310  312  336  343  348  350
  358  366  375  387  400  418  430  462  469  483  502  538  555  559
  562  564  614  623  626  647  649  674  694  706  708  761  778  796
  807  830  834  840  883  887  980  996 1007 1044 1064 1083 1084 1104
 1105 1111 1122 1176 1192 1257 1261 1265 1301 1312 1324 1327 1341 1375
 1472 1483 1492 1493 1496 1524 1534 1584 1592 1643 1646 1648 1675 1707
 1744 1779]


In [3]:
# STEP 1 - Compute confident joint

# Verify inputs
labels = np.asarray(labels)
pred_probs = np.asarray(pred_probs)

# Find the number of unique classes if K is not given
K = len(np.unique(labels))

# Estimate the probability thresholds for confident counting
# You can specify these thresholds yourself if you want
# as you may want to optimize them using a validation set.
# By default (and provably so) they are set to the average class prob.
thresholds = [np.mean(pred_probs[:, k][labels == k]) for k in range(K)]  # P(label^=k|label=k)
thresholds = np.asarray(thresholds)

# Compute confident joint
confident_joint = np.zeros((K, K), dtype=int)
for i, row in enumerate(pred_probs):
    s_label = labels[i]
    # Find out how many classes each example is confidently labeled as
    confident_bins = row >= thresholds - 1e-6
    num_confident_bins = sum(confident_bins)
    # If more than one conf class, inc the count of the max prob class
    if num_confident_bins == 1:
        confident_joint[s_label][np.argmax(confident_bins)] += 1
    elif num_confident_bins > 1:
        confident_joint[s_label][np.argmax(row)] += 1

# Normalize confident joint (use cleanlab, trust me on this)
confident_joint = cleanlab.count.calibrate_confident_joint(confident_joint, labels)

cleanlab.internal.util.print_joint_matrix(confident_joint)



 Joint Label Noise Distribution Matrix P(given_label, true_label) of shape (10, 10)
 p(s,y)	y=0	y=1	y=2	y=3	y=4	y=5	y=6	y=7	y=8	y=9
	---	---	---	---	---	---	---	---	---	---
s=0 |	168	4	1	0	0	4	2	0	2	0
s=1 |	4	163	3	1	3	1	0	0	4	1
s=2 |	0	3	153	5	0	1	0	1	3	1
s=3 |	0	0	5	167	0	1	2	3	6	1
s=4 |	1	2	0	0	165	3	0	0	0	3
s=5 |	4	2	5	1	3	160	1	1	1	8
s=6 |	1	3	1	1	1	1	167	3	1	5
s=7 |	0	0	0	0	3	3	1	167	0	1
s=8 |	4	3	3	1	6	3	4	4	157	1
s=9 |	1	0	3	1	4	3	0	4	3	160
	Trace(matrix) = 1627



In [4]:
# STEP 2 - Find label errors

# We arbitrarily choose at least 5 examples left in every class.
# Regardless of whether some of them might be label errors.
MIN_NUM_PER_CLASS = 5
# Leave at least MIN_NUM_PER_CLASS examples per class.
# NOTE prune_count_matrix is transposed (relative to confident_joint)
prune_count_matrix = cleanlab.filter._keep_at_least_n_per_class(
    prune_count_matrix=confident_joint.T,
    n=MIN_NUM_PER_CLASS,
)

s_counts = np.bincount(labels)
noise_masks_per_class = []
# For each row in the transposed confident joint
for k in range(K):
    noise_mask = np.zeros(len(pred_probs), dtype=bool)
    pred_probs_k = pred_probs[:, k]
    if s_counts[k] > MIN_NUM_PER_CLASS:  # Don't prune if not MIN_NUM_PER_CLASS
        for j in range(K):  # noisy label index (k is the true label index)
            if k != j:  # Only prune for noise rates, not diagonal entries
                num2prune = prune_count_matrix[k][j]
                if num2prune > 0:
                    # num2prune'th largest p(classk) - p(class j)
                    # for x with noisy label j
                    margin = pred_probs_k - pred_probs[:, j]
                    s_filter = labels == j
                    threshold = -np.partition(-margin[s_filter], num2prune - 1)[
                        num2prune - 1
                    ]
                    noise_mask = noise_mask | (s_filter & (margin >= threshold))
        noise_masks_per_class.append(noise_mask)
    else:
        noise_masks_per_class.append(np.zeros(len(labels), dtype=bool))

# Boolean label error mask
label_errors_bool = np.stack(noise_masks_per_class).any(axis=0)

# Remove label errors if given label == model prediction
for i, pred_label in enumerate(pred_probs.argmax(axis=1)):
    # np.all lets this work for multi_label and single label
    if label_errors_bool[i] and np.all(pred_label == labels[i]):
        label_errors_bool[i] = False

# Convert boolean mask to an ordered list of indices for label errors
label_errors_idx = np.arange(len(labels))[label_errors_bool]
# self confidence is the holdout probability that an example
# belongs to its given class label
self_confidence = np.array([np.mean(pred_probs[i][labels[i]]) for i in label_errors_idx])
margin = self_confidence - pred_probs[label_errors_bool].max(axis=1)
label_errors_idx = label_errors_idx[np.argsort(margin)]

print("Indices of label errors found by confident learning:")
print("Note label errors are sorted by likelihood of being an error")
print("but here we just sort them by index for comparison with above.")
print(np.array(sorted(label_errors_idx)))


Indices of label errors found by confident learning:
Note label errors are sorted by likelihood of being an error
but here we just sort them by index for comparison with above.
[   5    9   17   24   28   38   43   46   47   50   54   58   62   69
   77   85   87   92   94   95  105  107  110  111  129  155  157  159
  182  201  204  211  220  232  267  300  310  329  336  343  348  350
  358  375  387  418  421  430  446  462  480  483  496  521  528  538
  547  555  559  562  564  574  614  623  626  633  639  647  648  649
  658  674  685  694  700  703  706  708  723  735  737  757  761  766
  769  778  781  792  794  796  804  807  813  824  834  840  883  887
  920  965  980  985  996 1022 1044 1064 1077 1078 1083 1084 1104 1105
 1111 1118 1149 1176 1192 1229 1233 1241 1257 1261 1264 1265 1270 1274
 1288 1301 1312 1324 1327 1341 1375 1389 1472 1483 1492 1493 1522 1524
 1534 1540 1553 1572 1575 1584 1591 1592 1611 1646 1648 1658 1675 1685
 1690 1707 1714 1727 1729 1744 1747 1757 1

In [5]:
score = sum([e in label_errors_idx for e in actual_label_errors]) / NUM_ERRORS
print("% actual errors that confident learning found: {:.0%}".format(score))
score = sum([e in actual_label_errors for e in label_errors_idx]) / len(
    label_errors_idx
)
print("% confident learning errors that are actual errors: {:.0%}".format(score))


% actual errors that confident learning found: 87%
% confident learning errors that are actual errors: 53%
